In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
import corner
import sqlite3

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve
from justice.datasets import plasticc_data

In [ ]:
lcs = lightcurve.PlasticcDatasetLC.get_lc_by_target("data/plasticc_training_data.db",67)

In [ ]:
singlecurve = lcs[50]

In [ ]:
newxform = singlecurve.get_xform()

In [ ]:
values = newxform.as_array()

In [ ]:
values[0] = 300.

In [ ]:
# values

In [ ]:
# from tensorflow.contrib.framework import nest

In [ ]:
# generic_vals = [0., {b: 0. for b in singlecurve.expected_bands}, 1., {b: 1. for b in singlecurve.expected_bands}, 0.]
# vals = nest.flatten(generic_vals)

In [ ]:
# type(vals)

In [ ]:
newxform = singlecurve.get_xform(values)

In [ ]:
twocurve = newxform.transform(singlecurve)

In [ ]:
fig = vis.plot_lcs([singlecurve, twocurve])

In [ ]:
result = summ.opt_arclen(singlecurve, twocurve)

In [ ]:
vis.plot_arclen_res(singlecurve, twocurve, result)

In [ ]:
result

In [ ]:
singlecurve.bands['u'].flux

In [ ]:
import george
from george import kernels

x = singlecurve.bands['g'].time[:25]
y = singlecurve.bands['g'].flux[:25]
yerr = singlecurve.bands['g'].flux_err[:25]

kernel = np.mean(yerr) * kernels.Matern32Kernel(500.0, ndim=1)#, axes=0)
gp = george.GP(kernel)
gp.compute(x, yerr)

x_pred = np.linspace(min(x)-10, max(x)+10, 500)
pred, pred_var = gp.predict(y, x_pred, return_var=True)

plt.fill_between(x_pred, pred - np.sqrt(pred_var)+100., pred + np.sqrt(pred_var)+100.,
                color="k", alpha=0.2)
plt.plot(x_pred, pred+100., "k", lw=1.5, alpha=0.5)
plt.errorbar(x, y+100., yerr=yerr, fmt=".k", capsize=0)
plt.xlabel("x")
plt.ylabel("y")
plt.semilogy()